In [1]:
from IPython.display import clear_output
import os
import pandas as pd
import re
import yfinance as yf

In [2]:
stock_list = pd.read_csv('raw/stock_list.csv')['Stock']
dtype_dict = {stock: 'float64' for stock in stock_list}
raw_feature_dict = pd.read_excel(os.getcwd() + '/raw/bbg_data.xlsm', sheet_name=None, dtype=dtype_dict, skiprows=4)
raw_feature_dict.update(pd.read_excel(os.getcwd() + '/raw/Bloomberg20032024data_raw.xlsx', sheet_name=None, dtype=dtype_dict, skiprows=4))
feature_list = list(raw_feature_dict.keys())

Merge the dataset with one time index

In [3]:
feature_dict = {}
for feature in feature_list:
    df = raw_feature_dict[feature]
    columns_list = list(df.columns)
    for num, col in enumerate(columns_list):
        if num % 2 == 0:
            columns_list[num] = 'Date'
    df.columns = columns_list
    df = df.astype({'Date':'datetime64[ns]'})
    df_merge = pd.merge(df.iloc[:, :2].dropna(), df.iloc[:, 2:4].dropna(), on='Date', how='outer')
    for i in range(len(columns_list) // 2 - 2):
        df_merge = df_merge.merge(df.iloc[:, 2 * i + 4:2 * i + 6].dropna(), on='Date', how='outer')
    df_merge = df_merge.rename(columns=lambda x: re.sub(r'/', '-', x.split(' ')[0]))
    feature_dict[feature] = df_merge

In [4]:
start = '2012-12-31'
end = '2023-12-31'
code_list = stock_list.apply(lambda x: re.sub(r'/', '-', x.split(' ')[0])).rename('Code')
price_dict = {}
for code in code_list:
    price_df = yf.download(code, start=start, end=end, interval='1d')
    clear_output(wait=True)
    price_dict[code] = price_df

[*********************100%%**********************]  1 of 1 completed


Output the files to `.csv`

In [5]:
if not os.path.isdir('preprocessing'):
    os.mkdir('preprocessing')
    os.mkdir('preprocessing/feature')
    os.mkdir('preprocessing/price')

In [6]:
for feature in feature_list:
    feature_dict[feature].to_csv('preprocessing/feature/{}.csv'.format(feature))
for code in code_list:
    price_dict[code].to_csv('preprocessing/price/{}.csv'.format(code))